In [295]:
import re
import us
import feather
import pandas as pd
import gender_guesser.detector as d
import numpy as np
from nameparser import HumanName

In [179]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [180]:
kevin = feather.read_dataframe('data/cleaned_train_Kevin.feather')
kevin['year'] = kevin['raceyear'].apply(int)
kevin['district'] = kevin['district'].apply(int)
kevin = kevin.drop(columns='raceyear')

In [181]:
kevin

,state,district,repcandidate,demcandidate,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,log_med_income,percent_bachelor_or_above,year
0,Alabama,2,"Roby, Martha","Mathis, Nathan",0.5462,True,False,1,0.36979,0.40,False,True,10.646258,0.218,2016
1,Kansas,4,"Pompeo, Mike","Giroux, Daniel B.",0.6720,True,False,20,0.17527,0.40,False,True,10.860037,0.282,2016
2,California,5,"Santamaria, Carlos","Thompson, Mike",0.2313,False,True,6,0.37847,0.60,False,True,11.198297,0.304,2016
3,California,4,"McClintock, Tom","Derlet, Robert W.",0.6272,True,False,6,0.15606,0.60,False,True,11.212820,0.322,2016
4,California,3,"Cleek, N. Eugene","Garamendi, John",0.4065,False,True,6,0.33472,0.60,False,True,11.026923,0.245,2016
5,California,2,"Mensing, Dale K.","Huffman, Jared",0.2315,False,True,6,0.20327,0.60,False,True,11.106520,0.400,2016
6,California,1,"LaMalfa, Doug","Reed, Jim",0.5905,True,False,6,0.12946,NaN,None,True,10.768232,0.232,2016
7,New Hampshire,1,"Guinta, Frank C.","Shea-Porter, Carol",0.4923,True,False,33,0.06590,0.46,False,True,11.174329,0.361,2016
8,New Hampshire,2,"Lawrence, James","Kuster, Ann McLane",0.4767,False,True,33,0.06542,0.46,False,True,11.163708,0.348,2016
9,Kentucky,6,"Barr, Garland","Kemper, Nancy Jo",0.6109,True,False,21,0.15217,0.36,False,True,10.832872,0.303,2016


# 1, 4, 6, 10, 12, 13

In [355]:
# 1: percentage vote for democrat candidate in prev pres election
df1 = pd.read_csv('data/1_Scott&Akshay.csv')
df1 = df1.drop([445, 891, 1782]).reset_index(drop=True) # STATE_CD is NaN
vs = [re.split(r'(\d+)', str(s))[:-1] for s in list(df1['STATE_CD'])]
df1['state_fips'] = [int(us.states.lookup(x[0].strip()).fips) for x in vs]
df1['district'] = [int(x[1]) for x in vs]
df1 = df1.drop(columns=['Unnamed: 0', 'STATE_CD'])
df1['year'] = df1['year'].apply(int)
df1

,year,dem_percent,state_fips,district
0,2014,0.381725,1,1
1,2014,0.419044,1,2
2,2014,0.468938,1,3
3,2014,0.224703,1,4
4,2014,0.356304,1,5
5,2014,0.420412,1,6
6,2014,0.528870,1,7
7,2014,0.433689,4,1
8,2014,0.429787,4,2
9,2014,0.438798,4,3


In [365]:
df1 = df1.dropna()

In [402]:
joon.T

,0,1,2,3,4,5,6,7,8,9,...,9816,9817,9818,9819,9820,9821,9822,9823,9824,9825
state,Alabama,Kansas,California,California,California,California,California,New Hampshire,New Hampshire,Kentucky,...,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,West Virginia,West Virginia,West Virginia,Wyoming
district,2,4,5,4,3,2,1,1,2,6,...,6,5,4,3,2,1,3,2,1,0
rep_score,0.5462,0.672,0.2313,0.6272,0.4065,0.2315,0.5905,0.4923,0.4767,0.6109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rep_incumbent,True,True,False,True,False,False,True,True,False,True,...,True,True,False,False,False,False,False,True,True,True
dem_incumbent,False,False,True,False,True,True,False,False,True,False,...,False,False,True,True,True,False,False,False,False,False
state_fips,1,20,6,6,6,6,6,33,33,21,...,55,55,55,55,55,55,54,54,54,56
minority_percentage,0.36979,0.17527,0.37847,0.15606,0.33472,0.20327,0.12946,0.0659,0.06542,0.15217,...,0.07195,0.08459,0.49707,0.06477,0.14327,0.1289,0.06457,0.08572,0.05727,0.07973
pres_approval,0.4,0.4,0.6,0.6,0.6,0.6,NaN,0.46,0.46,0.36,...,0.56,0.56,0.56,0.56,0.56,0.56,0.33,0.33,0.33,0.34
same_as_pres_party_rep,False,False,False,False,False,False,None,False,False,False,...,True,True,True,True,True,True,True,True,True,True
same_as_pres_party_dem,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [367]:
joon = kevin.merge(df1, on=['year', 'state_fips', 'district'], how='left')

In [399]:
joon[joon['year'] >= 2010][['state', 'state_fips', 'district', 'year', 'dem_percent']].sort_values(by=['state_fips', 'district', 'year'],
                                                                             ascending=[1, 1, 0])

,state,state_fips,district,year,dem_percent
9398,Alabama,1,1,2018,0.352134
674,Alabama,1,1,2014,0.381725
9397,Alabama,1,2,2018,0.380884
0,Alabama,1,2,2016,0.419044
677,Alabama,1,2,2014,0.419044
1076,Alabama,1,2,2012,0.407024
1297,Alabama,1,2,2010,0.407024
9396,Alabama,1,3,2018,0.404996
126,Alabama,1,3,2016,0.468938
687,Alabama,1,3,2014,0.468938


In [498]:
final = joon

## BACKFILL

In [499]:
# backfill logic
state_fips = joon['state_fips'].unique()
for fip in state_fips:
    state_data = joon[joon['state_fips'] == fip]
    districts = state_data['district'].unique()
    for district in districts:
        district_data = state_data[state_data['district'] == district]
        dem_percent = district_data[['year', 'dem_percent']].sort_values(by=['year'], ascending=[0])
        dp = dem_percent.fillna(method='ffill')
        if dp.isnull().values.any(): # no value to forward-fill
            dp['dem_percent'] = [prior[fip]] * len(dp)
        final.iloc[dp.index]['dem_percent'] = dp['dem_percent']

/Users/joonyang/code/.virtualenvs/orca/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [438]:
prior = pd.read_csv('data/2016_republican_lean_percent.csv', encoding='utf-8')
prior['state_fips'] = [int(us.states.lookup(x.strip()).fips) for x in list(prior['state'])]
prior['dem_percent'] = prior['rep_percent'].apply(lambda x: 1 - float(x)/100)
prior = prior.set_index(keys=['state_fips'])
prior = prior.drop(columns=['state', 'rep_percent']).to_dict()['dem_percent']

In [496]:
joon['dem_percent'].isna().sum()

8430

In [509]:
final[final['state_fips'] == 2]

,state,district,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,...,year,dem_firstname,dem_lastname,rep_firstname,rep_lastname,dem_uncontested,rep_uncontested,dem_is_female,rep_is_female,dem_percent
267,Alaska,0,0.5828,True,False,2,0.35584,NaN,None,None,...,2016,Steve,Lindbeck,Don,Young,False,False,0.0,0.0,NaN
602,Alaska,0,0.5544,True,False,2,0.34361,NaN,False,True,...,2014,Forrest,Dunbar,Don,Young,False,False,0.0,0.0,NaN
1098,Alaska,0,0.6908,True,False,2,0.33481,NaN,False,True,...,2012,Sharon,Cissna,Don,Young,False,False,1.0,0.0,NaN
1333,Alaska,0,0.6932,True,False,2,0.32600,0.38,False,True,...,2010,Harry,Crawford,Don,Young,False,False,0.0,0.0,NaN
1851,Alaska,0,0.5272,True,False,2,0.32569,NaN,None,None,...,2008,Ethan,Berkowitz,Don,Young,False,False,0.0,0.0,NaN
2021,Alaska,0,0.5857,True,False,2,0.30887,NaN,None,None,...,2006,Diane,Benson,Don,Young,False,False,1.0,0.0,NaN
2291,Alaska,0,0.7607,True,False,2,NaN,NaN,None,None,...,2004,Thomas,Higgins,Don,Young,False,False,0.0,0.0,NaN
2753,Alaska,0,0.8117,True,False,2,NaN,NaN,None,None,...,2002,Clifford,Greene,Don,Young,False,False,0.0,0.0,NaN
3036,Alaska,0,0.8079,True,False,2,NaN,NaN,None,None,...,2000,Clifford,Greene,Don,Young,False,False,0.0,0.0,NaN
3571,Alaska,0,0.6439,True,False,2,NaN,NaN,None,None,...,1998,Jim,Duncan,Don,Young,False,False,0.0,0.0,NaN


In [503]:
joon.iloc[v.index]#.tail(10).fillna(method='ffill')['dem_percent'].isnull().values.any()

,state,district,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,...,year,dem_firstname,dem_lastname,rep_firstname,rep_lastname,dem_uncontested,rep_uncontested,dem_is_female,rep_is_female,dem_percent
9397,Alabama,2,NaN,True,False,1,0.36979,0.34,True,False,...,2018,Tabitha,Isner,Martha,Roby,False,False,1.0,1.0,0.380884
0,Alabama,2,0.5462,True,False,1,0.36979,0.40,False,True,...,2016,Nathan,Mathis,Martha,Roby,False,False,0.0,1.0,0.419044
677,Alabama,2,0.6741,True,False,1,0.34606,0.33,False,True,...,2014,Erick,Wright,Martha,Roby,False,False,0.0,1.0,0.419044
1076,Alabama,2,0.6366,True,False,1,0.33997,0.36,False,True,...,2012,Therese,Ford,Martha,Roby,False,False,1.0,1.0,0.407024
1297,Alabama,2,0.5109,False,True,1,0.34800,0.41,False,True,...,2010,Bobby,Bright,Martha,Roby,False,False,0.0,1.0,0.407024
1884,Alabama,2,0.4969,False,False,1,0.35261,NaN,None,None,...,2008,Bobby,Bright,Jay,Love,False,False,0.0,0.0,NaN
2210,Alabama,2,0.6954,True,False,1,0.34189,NaN,None,None,...,2006,Charles,James,Terry,Everett,False,False,0.0,0.0,NaN
2300,Alabama,2,0.7151,True,False,1,NaN,NaN,None,None,...,2004,Charles,James,Terry,Everett,False,False,0.0,0.0,NaN
2803,Alabama,2,0.6996,True,False,1,NaN,NaN,None,None,...,2002,Charles,Woods,Terry,Everett,False,False,0.0,0.0,NaN
3024,Alabama,2,0.7004,True,False,1,NaN,NaN,None,None,...,2000,Charles,Woods,Terry,Everett,False,False,0.0,0.0,NaN


In [491]:
[prior[23]] * 10

[0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61]

In [421]:
prior.to_dict()

{'rep_percent': {0: 39,
  1: 40,
  2: 30,
  3: 26,
  4: 27,
  5: 32,
  6: 29,
  7: 33,
  8: 41,
  9: 28,
  10: 33,
  11: 44,
  12: 42,
  13: 38,
  14: 43,
  15: 33,
  16: 41,
  17: 37,
  18: 42,
  19: 45,
  20: 56,
  21: 52,
  22: 50,
  23: 48,
  24: 38,
  25: 39,
  26: 47,
  27: 40,
  28: 39,
  29: 45,
  30: 47,
  31: 50,
  32: 45,
  33: 45,
  34: 43,
  35: 49,
  36: 41,
  37: 51,
  38: 42,
  39: 37,
  40: 53,
  41: 56,
  42: 56,
  43: 39,
  44: 34,
  45: 30,
  46: 36,
  47: 34,
  48: 28,
  49: 52},
 'state_fips': {0: 23,
  1: 33,
  2: 50,
  3: 25,
  4: 44,
  5: 9,
  6: 36,
  7: 34,
  8: 42,
  9: 24,
  10: 10,
  11: 54,
  12: 39,
  13: 26,
  14: 18,
  15: 17,
  16: 55,
  17: 27,
  18: 19,
  19: 29,
  20: 38,
  21: 46,
  22: 31,
  23: 20,
  24: 51,
  25: 37,
  26: 45,
  27: 13,
  28: 12,
  29: 21,
  30: 47,
  31: 1,
  32: 28,
  33: 5,
  34: 22,
  35: 40,
  36: 48,
  37: 30,
  38: 4,
  39: 8,
  40: 16,
  41: 56,
  42: 49,
  43: 32,
  44: 35,
  45: 6,
  46: 41,
  47: 53,
  48: 15,
  49: 

In [ ]:
# 4: incumbents <= this is a duplicate dataset...
df4 = pd.read_csv('data/4_Incumbency Status Ashwin Ryan Hamish.csv')
# eg. Alaska and Wyoming only send 1 person
df4['District'] = df4['District'].str.replace('at-large', '0')
vs = [re.split(r'(\d+)', str(s))[:-1] for s in list(df4['District'])]
df4['state_fips'] = [int(us.states.lookup(x[0].strip()).fips) for x in vs]
df4['district'] = [int(x[1]) for x in vs]
df4 = df4.drop(columns=['District'])
df4['year'] = df4['Year'].apply(int)
df4 = df4.drop(columns=['Year', 'Candidate'])
df4

In [60]:
# 6: gender
df6 = pd.read_csv('data/6_features_Zhaohong_Shu.csv')
df6['District'] = df6['District'].str.replace('at-large1', '0')
vs = [re.split(r'(\d+)', str(s))[:-1] for s in list(df6['District'])]
valid_states = []
valid_districts = []
invalids = []
for i, x in enumerate(vs):
    try:
        valid_states.append(int(us.states.lookup(x[0].strip()).fips))
        valid_districts.append(int(x[1]))
    except AttributeError:
        invalids.append(i)
df6 = df6.drop(invalids).reset_index(drop=True)
df6['state_fips'], df6['district'] = valid_states, valid_districts
df6['year'] = df6['Year'].apply(int)
df6 = df6.drop(columns=['Year'])
df6['last_name'] = df6['Name'].str.split(" ").apply(lambda x: x[-1])
df6['is_female'] = df6['Gender'].apply(lambda x: x == 'female')
df6 = df6.drop(columns=['District', 'Name', 'Gender'])

In [78]:
df6['rep_lastname'] = df6['dem_lastname']
df6 = df6.drop(columns=['dem_lastname'])

In [68]:
joon['rep_lastname'] = joon['repcandidate'].str.split(",").apply(lambda x: x[0].strip())
joon['dem_lastname'] = joon['demcandidate'].str.split(",").apply(lambda x: x[0].strip())

In [81]:
df6['rep_is_female'] = df6['dem_is_female']
df6 = df6.drop(columns=['dem_is_female'])

In [242]:
df6

,state_fips,district,year,rep_lastname,rep_is_female
0,1,1,2010,Bonner,True
1,1,1,2010,Walter,False
2,1,2,2010,Roby,True
3,1,2,2010,Bright,False
4,1,3,2010,Rogers,False
5,1,3,2010,Segrest,False
6,1,4,2010,Brooks,False
7,1,4,2010,Raby,False
8,1,5,2010,Sewell,True
9,1,5,2010,Chamberlain,False


In [107]:
# joon = joon.drop(columns=['dem_lastname', 'rep_lastname'])
joon.iloc[:-100]

,state,district,repcandidate,demcandidate,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,log_med_income,year,dem_percent,dem_is_female,rep_is_female
0,Alabama,2,"Roby, Martha","Mathis, Nathan",0.5462,True,False,1,0.36979,0.40,False,True,10.646258,2016,0.419044,NaN,False
1,California,42,"Calvert, Ken","Sheridan, Tim",0.5882,True,False,6,0.38404,0.60,False,True,11.245098,2016,0.617835,NaN,NaN
2,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN
3,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN
4,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN
5,Louisiana,6,"Graves, Garret","Lieberman, Richard",0.8078,True,False,22,0.30164,0.43,False,True,10.989859,2016,0.437192,NaN,NaN
6,California,53,"Veltmeyer, James","Davis, Susan A.",0.3299,False,True,6,0.35177,0.60,False,True,NaN,2016,0.526027,NaN,NaN
7,California,52,"Gitsham, Denise","Peters, Scott",0.4347,False,True,6,0.30440,0.60,False,True,11.417009,2016,0.526027,NaN,NaN
8,California,51,"Hidalgo, Juan M. Jr.","Vargas, Juan",0.2725,False,True,6,0.33433,0.60,False,True,10.750192,2016,0.529505,NaN,NaN
9,California,50,"Hunter, Duncan L.","Malloy, Patrick",0.6345,False,False,6,0.24730,0.60,False,True,11.180134,2016,0.526027,NaN,NaN


In [83]:
# dem
# joon = joon.merge(df6, on=['year', 'state_fips', 'district', 'dem_lastname'], how='left')
# rep
joon = joon.merge(df6, on=['year', 'state_fips', 'district', 'rep_lastname'], how='left')

In [183]:
# 10: primary percentage
df10 = pd.read_csv('data/10_Amil Merchant, Varun Krishnan, Thomas Chang.csv')
df10 = df10.drop([1491, 1499, 1503, 1507, 1509]).reset_index(drop=True) # Convention not primary
df10['primary_percent'] = df10['PRIMARY %'].apply(lambda x: float(x[:-1]) / 100)
# unable to classify in district eg. DES, ILS, NYSFULL, NYSUN, WVS
blacklist = []
for i, row in df10.iterrows():
    if len(row['district']) > 4:
        try:
            int(row[i])
        except:   
            blacklist.append(i)
df10 = df10.drop(blacklist).reset_index(drop=True)
df10['state_fips'] = df10['district'].apply(lambda x: int(us.states.lookup(states[x[:2]]).fips))
df10['district'] = df10['district'].apply(lambda x: int((x.split('-')[0].strip())[2:]))
df10 = df10.drop(columns=['Unnamed: 0', 'PRIMARY %'])
df10.columns = ['district', 'candidate', 'party', 'year', 'primary_percent', 'state_fips']

In [184]:
R = df10[df10['party'] == 'R'].drop(columns=['candidate', 'party'])
D = df10[df10['party'] == 'D'].drop(columns=['candidate', 'party'])
R.columns = ['district', 'year', 'rep_primary_percent', 'state_fips']
D.columns = ['district', 'year', 'dem_primary_percent', 'state_fips']

In [122]:
joon = joon.merge(R, on=['year', 'state_fips', 'district'], how='left')
joon = joon.merge(D, on=['year', 'state_fips', 'district'], how='left')
joon

,state,district,repcandidate,demcandidate,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,log_med_income,year,dem_percent,dem_is_female,rep_is_female,rep_primary_percent,dem_primary_percent
0,Alabama,2,"Roby, Martha","Mathis, Nathan",0.5462,True,False,1,0.36979,0.40,False,True,10.646258,2016,0.419044,NaN,False,1.0000,1.0000
1,California,42,"Calvert, Ken","Sheridan, Tim",0.5882,True,False,6,0.38404,0.60,False,True,11.245098,2016,0.617835,NaN,NaN,0.6749,0.1581
2,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN
3,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN
4,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN
5,Louisiana,6,"Graves, Garret","Lieberman, Richard",0.8078,True,False,22,0.30164,0.43,False,True,10.989859,2016,0.437192,NaN,NaN,NaN,NaN
6,California,53,"Veltmeyer, James","Davis, Susan A.",0.3299,False,True,6,0.35177,0.60,False,True,NaN,2016,0.526027,NaN,NaN,0.2067,0.5626
7,California,52,"Gitsham, Denise","Peters, Scott",0.4347,False,True,6,0.30440,0.60,False,True,11.417009,2016,0.526027,NaN,NaN,0.3526,0.4229
8,California,51,"Hidalgo, Juan M. Jr.","Vargas, Juan",0.2725,False,True,6,0.33433,0.60,False,True,10.750192,2016,0.529505,NaN,NaN,0.3130,0.6834
9,California,50,"Hunter, Duncan L.","Malloy, Patrick",0.6345,False,False,6,0.24730,0.60,False,True,11.180134,2016,0.526027,NaN,NaN,0.7043,0.2434


In [185]:
new = kevin.merge(R, on=['year', 'state_fips', 'district'], how='left')
new = kevin.merge(D, on=['year', 'state_fips', 'district'], how='left')

In [186]:
new

,state,district,repcandidate,demcandidate,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,log_med_income,percent_bachelor_or_above,year,dem_primary_percent
0,Alabama,2,"Roby, Martha","Mathis, Nathan",0.5462,True,False,1,0.36979,0.40,False,True,10.646258,0.218,2016,1.0000
1,Kansas,4,"Pompeo, Mike","Giroux, Daniel B.",0.6720,True,False,20,0.17527,0.40,False,True,10.860037,0.282,2016,1.0000
2,California,5,"Santamaria, Carlos","Thompson, Mike",0.2313,False,True,6,0.37847,0.60,False,True,11.198297,0.304,2016,0.8044
3,California,4,"McClintock, Tom","Derlet, Robert W.",0.6272,True,False,6,0.15606,0.60,False,True,11.212820,0.322,2016,NaN
4,California,3,"Cleek, N. Eugene","Garamendi, John",0.4065,False,True,6,0.33472,0.60,False,True,11.026923,0.245,2016,0.5348
5,California,2,"Mensing, Dale K.","Huffman, Jared",0.2315,False,True,6,0.20327,0.60,False,True,11.106520,0.400,2016,0.6791
6,California,1,"LaMalfa, Doug","Reed, Jim",0.5905,True,False,6,0.12946,NaN,None,True,10.768232,0.232,2016,0.3015
7,New Hampshire,1,"Guinta, Frank C.","Shea-Porter, Carol",0.4923,True,False,33,0.06590,0.46,False,True,11.174329,0.361,2016,0.9811
8,New Hampshire,2,"Lawrence, James","Kuster, Ann McLane",0.4767,False,True,33,0.06542,0.46,False,True,11.163708,0.348,2016,0.9861
9,Kentucky,6,"Barr, Garland","Kemper, Nancy Jo",0.6109,True,False,21,0.15217,0.36,False,True,10.832872,0.303,2016,0.6087


In [187]:
# 12: labor force
df12 = pd.read_csv('data/12_employment_16_to_64 Peter Shen _ Harry Burke.csv')
vs = [re.split(r'(\d+)', str(s))[:-1] for s in list(df12['StateDistrict'])]
df12['state_fips'] = [int(us.states.lookup(x[0].strip()).fips) for x in vs]
df12['district'] = [int(x[1]) for x in vs]
df12['year'] = df12['Year'] + 2000
df12 = df12.drop(columns=['StateDistrict', 'Year', 'Name'])
df12.columns = ['labor_force', 'state_fips', 'district', 'year']

In [189]:
new = new.merge(df12, on=['year', 'state_fips', 'district'], how='left')

In [130]:
joon = joon.merge(df12, on=['year', 'state_fips', 'district'], how='left')
joon

,state,district,repcandidate,demcandidate,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,log_med_income,year,dem_percent,dem_is_female,rep_is_female,rep_primary_percent,dem_primary_percent,labor_force
0,Alabama,2,"Roby, Martha","Mathis, Nathan",0.5462,True,False,1,0.36979,0.40,False,True,10.646258,2016,0.419044,NaN,False,1.0000,1.0000,68.10000
1,California,42,"Calvert, Ken","Sheridan, Tim",0.5882,True,False,6,0.38404,0.60,False,True,11.245098,2016,0.617835,NaN,NaN,0.6749,0.1581,72.70000
2,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN,NaN
3,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN,NaN
4,Louisiana,4,"Johnson, Mike","Jones, Marshall",0.4669,False,False,22,0.39736,0.43,False,True,10.590315,2016,0.398072,NaN,NaN,NaN,NaN,NaN
5,Louisiana,6,"Graves, Garret","Lieberman, Richard",0.8078,True,False,22,0.30164,0.43,False,True,10.989859,2016,0.437192,NaN,NaN,NaN,NaN,69.90000
6,California,53,"Veltmeyer, James","Davis, Susan A.",0.3299,False,True,6,0.35177,0.60,False,True,NaN,2016,0.526027,NaN,NaN,0.2067,0.5626,69.60000
7,California,52,"Gitsham, Denise","Peters, Scott",0.4347,False,True,6,0.30440,0.60,False,True,11.417009,2016,0.526027,NaN,NaN,0.3526,0.4229,68.00000
8,California,51,"Hidalgo, Juan M. Jr.","Vargas, Juan",0.2725,False,True,6,0.33433,0.60,False,True,10.750192,2016,0.529505,NaN,NaN,0.3130,0.6834,68.30000
9,California,50,"Hunter, Duncan L.","Malloy, Patrick",0.6345,False,False,6,0.24730,0.60,False,True,11.180134,2016,0.526027,NaN,NaN,0.7043,0.2434,69.40000


In [49]:
# 13: Special elections <- only 43 data points... too sparse
df13 = pd.read_csv('data/13_Benedict Brady_Special_Elections_Feature.csv')
df13['District'][35] = 'Montana 0'
vs = [re.split(r'(\d+)', str(s))[:-1] for s in list(df13['District'])] 
df13['state_fips'] = [us.states.lookup(x[0][:-1]).fips for x in vs]
df13['district'] = [int(x[1]) for x in vs]
df13

/Users/joonyang/code/.virtualenvs/orca/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,District,Congress,Date,Predecessor,Predecessor Party,Winner,Winner Party,Winner Vote Share,Runner Up Vote Share,state_fips,district
0,Florida 19,2010,13-Apr-10,Robert Wexler,D,Ted Deutch,D,62.1,35.1,12,19
1,Pennsylvania 12,2010,18-May-10,John Murtha,D,Mark Critz,D,52.6,45.1,42,12
2,Hawaii 1,2010,22-May-10,Neil Abercrombie,D,Charles Djou,R,39.4,30.8,15,1
3,Georgia 9,2010,8-Jun-10,Nathan Deal,R,Tom Graves,R,56.5,43.5,13,9
4,Indiana 3,2010,2-Nov-10,Mark Souder,R,Marlin Stutzman,R,63.0,33.0,18,3
5,New York 29,2010,2-Nov-10,Eric Massa,D,Tom Reed,R,56.0,44.0,36,29
6,New York 26,2012,24-May-11,Chris Lee,R,Kathy Hochul,D,47.2,43.2,36,26
7,California 36,2012,12-Jul-11,Jane Harman,D,Janice Hahn,D,54.9,45.1,06,36
8,Nevada 2,2012,13-Sep-11,Dean Heller,R,Mark Amodei,R,58.0,36.0,32,2
9,New York 9,2012,13-Sep-11,Anthony Weiner,D,Bob Turner,R,51.7,46.6,36,9


In [246]:
df6.columns = ['state_fips', 'district', 'year', 'dem_lastname', 'dem_is_female']

In [339]:
kevin = feather.read_dataframe('data/kevin_joint.feather')
kevin['year'] = kevin['raceyear'].apply(int)
kevin['district'] = kevin['district'].apply(int)
kevin = kevin.drop(columns='raceyear')

In [340]:
dem_first, dem_last = [], []
rep_first, rep_last = [], []
dem_uncontested, rep_uncontested = [], []
for i, row in kevin.iterrows():
    if row['demcandidate']:
        dem_name = HumanName(str(row['demcandidate']))
        if len(dem_name['first']) < 3:
            dem_first.append(dem_name['middle'])
        else:
            dem_first.append(dem_name['first'])
        dem_last.append(dem_name['last'])
        rep_uncontested.append(False)
    else:
        dem_first.append(np.nan)
        dem_last.append(np.nan)
        rep_uncontested.append(True)
    if row['repcandidate']:
        rep_name = HumanName(str(row['repcandidate']))    
        if len(rep_name['first']) < 3:
            rep_first.append(rep_name['middle'])
        else: 
            rep_first.append(rep_name['first'])
        rep_last.append(rep_name['last'])
        dem_uncontested.append(False)
    else:
        rep_first.append(np.nan)
        rep_last.append(np.nan)
        dem_uncontested.append(True)
kevin['dem_firstname'] = dem_first
kevin['dem_lastname'] = dem_last
kevin['rep_firstname'] = rep_first
kevin['rep_lastname'] = rep_last
kevin['dem_uncontested'] = dem_uncontested
kevin['rep_uncontested'] = rep_uncontested
kevin = kevin.drop(columns=['demcandidate', 'repcandidate'])

In [342]:
# dem
kevin = kevin.merge(df6, on=['state_fips', 'district', 'year', 'dem_lastname'], how='left')

In [343]:
df6.columns = ['state_fips', 'district', 'year', 'rep_lastname', 'rep_is_female']

In [344]:
# rep
kevin = kevin.merge(df6, on=['state_fips', 'district', 'year', 'rep_lastname'], how='left')

In [345]:
kevin

,state,district,rep_score,rep_incumbent,dem_incumbent,state_fips,minority_percentage,pres_approval,same_as_pres_party_rep,same_as_pres_party_dem,...,fivethirtyeight_district,year,dem_firstname,dem_lastname,rep_firstname,rep_lastname,dem_uncontested,rep_uncontested,dem_is_female,rep_is_female
0,Alabama,2,0.5462,True,False,1,0.36979,0.40,False,True,...,None,2016,Nathan,Mathis,Martha,Roby,False,False,NaN,False
1,Kansas,4,0.6720,True,False,20,0.17527,0.40,False,True,...,None,2016,Daniel,Giroux,Mike,Pompeo,False,False,NaN,False
2,California,5,0.2313,False,True,6,0.37847,0.60,False,True,...,None,2016,Mike,Thompson,Carlos,Santamaria,False,False,NaN,NaN
3,California,4,0.6272,True,False,6,0.15606,0.60,False,True,...,None,2016,Robert,Derlet,Tom,McClintock,False,False,NaN,False
4,California,3,0.4065,False,True,6,0.33472,0.60,False,True,...,None,2016,John,Garamendi,Eugene,Cleek,False,False,NaN,NaN
5,California,2,0.2315,False,True,6,0.20327,0.60,False,True,...,None,2016,Jared,Huffman,Dale,Mensing,False,False,NaN,NaN
6,California,1,0.5905,True,False,6,0.12946,NaN,None,True,...,None,2016,Jim,Reed,Doug,LaMalfa,False,False,NaN,NaN
7,New Hampshire,1,0.4923,True,False,33,0.06590,0.46,False,True,...,None,2016,Carol,Shea-Porter,Frank,Guinta,False,False,False,False
8,New Hampshire,2,0.4767,False,True,33,0.06542,0.46,False,True,...,None,2016,Ann,Kuster,James,Lawrence,False,False,False,NaN
9,Kentucky,6,0.6109,True,False,21,0.15217,0.36,False,True,...,None,2016,Nancy,Kemper,Garland,Barr,False,False,NaN,False


In [296]:
detector = d.Detector()

In [346]:
dem_is_female, rep_is_female = [], []
c = 0
for i, row in kevin.iterrows():
    dverdict = detector.get_gender(row['dem_firstname'])
    if 'female' in dverdict:
        c += 1
        dem_is_female.append(1)
    elif 'male' in dverdict:
        dem_is_female.append(0)
    else:
        dem_is_female.append(0.5)
    rverdict = detector.get_gender(row['rep_firstname'])
    if 'female' in rverdict:
        c += 1
        rep_is_female.append(1)
    elif 'male' in rverdict:
        rep_is_female.append(0)
    else:
        rep_is_female.append(0.5)
print(c/len(kevin))

0.24852432322409934


In [348]:
kevin['dem_is_female'] = dem_is_female
kevin['rep_is_female'] = rep_is_female

In [354]:
kevin.T

,0,1,2,3,4,5,6,7,8,9,...,9816,9817,9818,9819,9820,9821,9822,9823,9824,9825
state,Alabama,Kansas,California,California,California,California,California,New Hampshire,New Hampshire,Kentucky,...,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,Wisconsin,West Virginia,West Virginia,West Virginia,Wyoming
district,2,4,5,4,3,2,1,1,2,6,...,6,5,4,3,2,1,3,2,1,0
rep_score,0.5462,0.672,0.2313,0.6272,0.4065,0.2315,0.5905,0.4923,0.4767,0.6109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rep_incumbent,True,True,False,True,False,False,True,True,False,True,...,True,True,False,False,False,False,False,True,True,True
dem_incumbent,False,False,True,False,True,True,False,False,True,False,...,False,False,True,True,True,False,False,False,False,False
state_fips,1,20,6,6,6,6,6,33,33,21,...,55,55,55,55,55,55,54,54,54,56
minority_percentage,0.36979,0.17527,0.37847,0.15606,0.33472,0.20327,0.12946,0.0659,0.06542,0.15217,...,0.07195,0.08459,0.49707,0.06477,0.14327,0.1289,0.06457,0.08572,0.05727,0.07973
pres_approval,0.4,0.4,0.6,0.6,0.6,0.6,NaN,0.46,0.46,0.36,...,0.56,0.56,0.56,0.56,0.56,0.56,0.33,0.33,0.33,0.34
same_as_pres_party_rep,False,False,False,False,False,False,None,False,False,False,...,True,True,True,True,True,True,True,True,True,True
same_as_pres_party_dem,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
